In [303]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in tqdm(os.listdir(maindir + '\\testing_modules')):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)
plt.close('All')

100%|██████████| 20/20 [00:00<00:00, 83.13it/s]


In [2]:
%matplotlib qt
import skimage.io as io
import xarray as xr
from lmfit import Model
from lmfit.models import ConstantModel

In [3]:
#scanid = 149248
scanid = 149566
filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
#filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'

test = XRDMap.from_image_stack(map_filename, wd=filedir, energy=15, scanid=scanid)
test.map.images[test.map.images < 0] = 0
test.map.correct_images()
test.set_calibration('scan149562_dexela_calibration.poni')
#test.set_calibration('scan149256_dexela_calibration.poni')

Estimating dark-field with minimum method.
Correcting dark-field...done!
Estimating flat-field from median values.
Correcting flat-field...done!
Image corrections complete!
Compressing and writing processed images to disk...done!


In [4]:
test.map.plot_image()

In [491]:
test.map.plot_image(test.map.composite_image)

In [103]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
for filename, phase_name in zip(['AMCSD\\Hematite_0017806.cif',
                                 'AMCSD\\Iron-alpha_0011214.cif',
                                 'AMCSD\\Iron-beta_0011215.cif',
                                 'AMCSD\\Iron-delta_0011216.cif',
                                 'AMCSD\\Iron_0011146.cif',
                                 'AMCSD\\Cementite_0013523.cif',
                                 'AMCSD\\Zincite_0011555.cif'],
                                ['Hematite', 'Iron-ferrite', 'Iron-beta',
                                 'Iron-delta', 'Iron-austenite', 'Cementite',
                                 'Zincite']):
    
    test.load_phase(filename, filedir=cif_dir, phase_name=phase_name)

In [5]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
test.load_phase('Barker\\1521772COD.cif', filedir=cif_dir, phase_name="LiNbO3 161")
test.load_phase('Barker\\LiNbO3.cif', filedir=cif_dir, phase_name="LiNbO3 148")
test.load_phase('AMCSD\\Platinum_0011157.cif', filedir=cif_dir, phase_name="Platinum")
test.load_phase('AMCSD\\Calcite_0000984.cif', filedir=cif_dir, phase_name="Calcite")
test.load_phase('AMCSD\\Aluminum_0011136.cif', filedir=cif_dir, phase_name="Aluminum")
test.load_phase('AMCSD\\Zincite_0011555.cif', filedir=cif_dir, phase_name="Zincite")
test.load_phase('AMCSD\\Gold_0011140.cif', filedir=cif_dir, phase_name="Gold")
test.load_phase('AMCSD\\Chromium_0011209.cif', filedir=cif_dir, phase_name="Chromium BCC")
test.load_phase('AMCSD\\Chromium_0011170.cif', filedir=cif_dir, phase_name="Chromium HCP")
test.load_phase('AMCSD\\Chromium_0011144.cif', filedir=cif_dir, phase_name="Chromium FCC")

XU.materials: Warning: element O used instead of O-2
XU.materials: Warning: element Nb used instead of Nb+5
XU.materials: Warning: element Li used instead of Li+1


In [6]:
test.select_phases(remove_less_than=0.1)

In [7]:
test.calibrate_images()

Composite of processed images not saved. Creating composite.
Calibrating images...

  0%|          | 0/36 [00:00<?, ?it/s]

done!
Compressing and writing calibrated images to disk.
This may take awhile...done!


In [111]:
selected_images = np.median(test.map.images[::5, ::5], axis=(0, 1)) # Randomize for less that certain number of images...
bkg_noise = np.std(selected_images[selected_images != 0]) 
def dask_find_blobs(pixel):
    thresh = np.median(pixel[pixel != 0]) + 3 * bkg_noise
    blob_df = find_blobs(pixel, thresh=thresh, sigma=3, blob_expansion=5,
                     tth=test.tth, chi=test.chi, plotme=False, find_contours=False,
                     min_blob_significance=10 * bkg_noise / test.tth_resolution / test.chi_resolution)
    return blob_df

In [112]:
iterable = test.map.images.reshape(test.map.num_images, len(test.chi), len(test.tth))
blob_df_lst = parallel_loop(dask_find_blobs, iterable)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

In [261]:
 # Randomize for less that certain number of images...
#i = np.random.randint(test.map.images.shape[0])
#j = np.random.randint(test.map.images.shape[1])
#i, j = 1, 2

selected_images = np.copy(test.map.images[::5, ::5]) # Randomize for less that certain number of images...
bkg_noise = np.std(selected_images[selected_images != 0]) 

#pixel = test.map.images.reshape(test.map.num_images, len(test.chi), len(test.tth))[1]
pixel = test.map.images[i, j]
#pixel = test.map.images[1, 0]
thresh = np.median(pixel[pixel != 0]) + 3 * bkg_noise
blob_df = find_blobs(pixel, thresh=thresh, sigma=3, blob_expansion=5,
                    tth=test.tth, chi=test.chi, plotme=True, find_contours=True,
                    min_blob_significance=10 * bkg_noise / test.tth_resolution / test.chi_resolution)
print(len(blob_df))

58


In [161]:
blob_df

,intensity,area,significance,values,mask,max_coords,center_of_mass,contour
0,8.639657,1.130270,7.643886,"[0.005503828, 0.004137438, 0.0045799427, 0.004...","[[275, 437], [275, 438], [275, 439], [275, 440...","[21.05615091209026, 21.11013842282641]","[21.05012320928043, 21.09861947633619]","[[21.076350700965037, 21.05615091209026, 21.03..."
1,5.236819,0.842342,6.216972,"[0.0030591122, 0.0043983376, 0.0048938454, 0.0...","[[796, 1301], [796, 1302], [796, 1303], [796, ...","[38.48856871102365, -5.390368278415558]","[38.477001608371566, -5.065150918472483]","[[38.50876849989844, 38.48856871102365, 38.468..."
2,2.746269,0.486006,5.650688,"[0.0045928545, 0.0047189435, 0.004675045, 0.00...","[[815, 652], [815, 653], [815, 654], [815, 655...","[25.338506153543193, -5.84466267900828]","[25.33209991248583, -5.714664025048965]","[[25.35870594241797, 25.338506153543193, 25.31..."


In [8]:
def spot_search(image, bkg_noise, multiplier=3, sigma=3, plotme=False):
    mask_thresh = np.median(image[image != 0]) + multiplier * bkg_noise
    thresh_img = gaussian_filter(median_filter(image, size=1), sigma=sigma)
    mask = thresh_img > mask_thresh

    #spot_img = gaussian_filter(median_filter(pixel, size=1), sigma=5)
    spots = peak_local_max(thresh_img,
                           threshold_rel=multiplier * bkg_noise,
                           min_distance=2, # Just a pinch more spacing
                           labels=mask,
                           num_peaks_per_label=np.inf)
    
    if plotme:
        fig, ax = plt.subplots(1, 1, figsize=(9, 6), dpi=200)

        im = ax.imshow(image, vmin=0, vmax=10 * mask_thresh)
        fig.colorbar(im, ax=ax)
        ax.scatter(spots[:, 1], spots[:, 0], s=1, c='r')

        plt.show()

    return spots, mask

In [98]:
from skimage.measure import label

i = np.random.randint(test.map.images.shape[0])
j = np.random.randint(test.map.images.shape[1])
i, j = 2, 3

selected_images = np.median(np.copy(test.map.images), axis=(0, 1)) # Randomize for less that certain number of images...
bkg_noise = np.std(selected_images[selected_images != 0])

spots, mask = spot_search(test.map.images[i, j], bkg_noise, plotme=True, multiplier=10)
blob_img = label(mask)
print(len(spots))

623


In [99]:
image = test.map.images[i, j]

x_coords, y_coords = np.meshgrid(test.tth, test.chi[::-1])

In [100]:
from skimage.segmentation import watershed
from scipy import ndimage as ndi

# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt([blob_img != 0][0])
coords = peak_local_max(distance, footprint=np.ones((3, 3)),
                        labels=[blob_img != 0][0], num_peaks_per_label=np.inf,
                        threshold_rel=0.5)
coords = spots
print(len(coords))
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)
labels = watershed(-distance, markers, mask=[blob_img != 0][0])

fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow([blob_img != 0][0], cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')
ax[2].scatter(spots[:, 1], spots[:, 0], s=1, c='r')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

print(len(np.unique(labels)))

623
624


In [101]:
def label_nearest_spots(spots, max_dist=25, max_neighbors=np.inf):
    # Redo to account for different distance values. Currently using pixel differences
    from sklearn.metrics.pairwise import euclidean_distances
    dist = euclidean_distances(spots)

    spot_indices = list(range(len(spots)))

    # Ignore distances greater than bounds
    # Ignore same pairs
    dist[dist > max_dist] = np.nan
    #dist[dist == 0] = np.nan

    # Create dataset
    data = np.empty((len(spots), 3))
    data[:] = np.nan
    data[:, :-1] = spots

    labels=[0]
    for i in spot_indices:
        # This seems unecessary...
        connections = dist[i] < max_dist

        # Useful for connectivity. Not useful for current implementation
        if sum(connections) > max_neighbors:
            connections = dist[i] <= sorted(dist[i][connections])[max_neighbors - 1]
        
        # Find lowest label is group of connected points
        if np.all(np.isnan(data[connections][:, -1])):
            data[connections, -1] = labels[-1]
            #for index, con in enumerate(connections):
            #    if con:
            #        data[index, -1] = labels[-1]
            labels.append(labels[-1] + 1)
            #print('New label added')
        else:
            base_label = np.nanmin(data[connections][:, -1])
            data[connections, -1] = base_label
            #for index, con in enumerate(connections):
            #    if con:
            #        data[index, -1] = base_label
            #print('Connecting to other label')
        
        # Relabel previous labels if connected with others...
        extra_labels = np.unique(data[connections][:, -1])
        if np.any(extra_labels != labels[-1]):
            for extra in extra_labels:
                if extra != labels[-1]:
                    data[data[:, -1] == extra][-1] = labels[-1]

    data = data.astype(np.int32)
    labels = np.array(labels).astype(np.int32)
    # labels is not perfectly sequential. Why???
    return data

In [304]:
data = label_nearest_spots(spots, max_dist=20, max_neighbors=np.inf)
print(len(np.unique(data[:, -1])))

new_spots = []

for label in np.unique(data[:, -1]):
    if label in data[:, -1]:
        new_spot = np.mean(data[data[:, -1] == label][:, :-1], axis=0)
        new_spot = np.round(new_spot, 0).astype(np.int32)
        new_spots.append(new_spot)

new_spots = np.asarray(new_spots)

399


In [305]:
from skimage.segmentation import watershed
from scipy import ndimage as ndi

# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt([blob_img != 0][0])
coords = peak_local_max(distance, footprint=np.ones((3, 3)),
                        labels=[blob_img != 0][0], num_peaks_per_label=np.inf,
                        threshold_rel=0.5)
coords = np.asarray(new_spots)
print(len(coords))
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)
labels = watershed(-distance, markers, mask=[blob_img != 0][0])
# I don't like this, but it fixes some labeling issues
labels = median_filter(labels, size=3)

fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow([blob_img != 0][0], cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')
ax[2].scatter(spots[:, 1], spots[:, 0], s=1, c='r')
#ax[2].scatter(new_spots[:, 1], new_spots[:, 0], s=1, c='r')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

print(len(np.unique(labels)))

399
346


In [306]:
for spot in spots:
    if labels[*spot] == 0:
        labels[blob_img == blob_img[*spot]] = (np.max(labels) + 1)
    
blob_img = labels

In [326]:
PeakModel = GaussianFunctions()

# Add watershed to further separate blobs...
spot_labels = np.array([blob_img[*spot] for spot in spots])
full_popt = []
full_residual = 0

for blob in np.unique(blob_img):
    if blob == 0:
        continue

    num_spots = len(spots[spot_labels == blob])
    if num_spots == 0:
        print(f'No spots in blob {blob}. Proceeding to next blob.')
        continue

    bkg_mask = [blob_img != 0][0]
    blob_mask = [blob_img[bkg_mask] == blob][0]
    blob_size = np.sum(blob_mask)

    if blob_size / (6 * num_spots) <= 1:
        print(f'More unknowns than pixels in blob {blob}!')
        print('Proceeding to next blob.')
        continue


    print(f'Fitting {len(spots[spot_labels == blob])} spots within blob {blob}...', end='', flush=True)

    blob_int = image[bkg_mask][blob_mask]
    blob_x = x_coords[bkg_mask][blob_mask]
    blob_y = y_coords[bkg_mask][blob_mask]

    p0 = [np.median(image)]
    for i, spot in enumerate(spots[spot_labels == blob]):
        p0.append(image[*spot]) # amp
        p0.append(x_coords[*spot]) # x0
        p0.append(y_coords[*spot]) # y0
        p0.append(0.05) # sigma_x
        p0.append(0.1) # sigma_y
        p0.append(0) # theta

    bounds = generate_bounds(p0[1:], PeakModel.func_2d, tth_step=None, chi_step=None)
    bounds[0].insert(0, np.min(image))
    bounds[1].insert(0, np.max(image))

    popt, pcov = curve_fit(PeakModel.multi_2d, [blob_x, blob_y], blob_int, p0=p0, bounds=bounds)
    r_squared = compute_r_squared(blob_int, PeakModel.multi_2d([blob_x, blob_y], *popt))
    residual = blob_int - PeakModel.multi_2d([blob_x, blob_y], *popt)
    print('done!')
    print(f'Fitting R²: {r_squared:.4f}')


    full_popt.extend(popt[1:])
    full_residual += np.sum(residual**2)
full_popt = np.asarray(full_popt)
print('Done!')

Fitting 1 spots within blob 1...done!
Fitting R²: 0.8993
No spots in blob 2. Proceeding to next blob.
Fitting 1 spots within blob 3...done!
Fitting R²: 0.8921
Fitting 1 spots within blob 4...done!
Fitting R²: 0.8454
Fitting 3 spots within blob 5...done!
Fitting R²: 0.9581
Fitting 1 spots within blob 6...done!
Fitting R²: 0.9585
Fitting 3 spots within blob 7...done!
Fitting R²: 0.8877
Fitting 1 spots within blob 8...done!
Fitting R²: 0.7921
Fitting 1 spots within blob 9...done!
Fitting R²: 0.9671
Fitting 3 spots within blob 11...done!
Fitting R²: 0.7805
No spots in blob 12. Proceeding to next blob.
No spots in blob 13. Proceeding to next blob.
Fitting 1 spots within blob 14...done!
Fitting R²: 0.9448
No spots in blob 15. Proceeding to next blob.
Fitting 2 spots within blob 16...done!
Fitting R²: 0.9197
Fitting 5 spots within blob 17...done!
Fitting R²: 0.8028
Fitting 2 spots within blob 19...done!
Fitting R²: 0.8297
Fitting 2 spots within blob 20...done!
Fitting R²: 0.8920
Fitting 1 spo

In [327]:
new_popt = full_popt.reshape(len(full_popt) // 6, 6)
trimmed_popt = new_popt[new_popt[:, 0] > np.median(image[image != 0]) + 10 * bkg_noise].flatten()
fit_popt = list(trimmed_popt)
fit_popt.insert(0, np.median(image[image != 0]))

In [328]:
fit_img = PeakModel.multi_2d([x_coords.ravel(), y_coords.ravel()], *fit_popt)

In [329]:
test.map.plot_image(fit_img.reshape(test.calibrated_shape))
#test.map.plot_image(image)

In [401]:
popt[0]

0.00612754375535511

In [330]:
bkg_mask = (blob_img != 0)

In [331]:
residual = image[bkg_mask] - fit_img.reshape(test.calibrated_shape)[bkg_mask]

In [332]:
np.sum(residual ** 2)

391.09165751429344

In [389]:
np.ones(591).shape

(591,)

In [396]:
(offsets.reshape(*offsets.shape, 1)).shape

(591, 1)

In [399]:
[1,] * 10

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [380]:
spot_popt.shape

(591, 6)

In [359]:
df_labels = ['fit_amp', 'fit_tth', 'fit_chi', 'fit_sigma_x', 'fit_sigma_y', 'fit_theta']
spot_popt = np.array(fit_popt[1:]).reshape(len(fit_popt[1:]) // 6, 6)

spot_dict = dict(zip(df_labels, spot_popt.T))

spot_df = pd.DataFrame.from_dict(spot_dict)

In [360]:
spot_df

,fit_amp,fit_tth,fit_chi,fit_sigma_x,fit_sigma_y,fit_theta
0,0.441586,17.398186,30.757755,0.052457,0.212020,-6.300719
1,0.068245,18.430066,28.803934,0.040462,0.304032,-2.632409
2,0.044340,18.406530,27.363748,0.075470,0.530047,1.768484
3,0.454172,17.371811,27.360101,0.061029,0.404975,-0.756163
4,0.391349,17.453968,26.404019,0.044833,0.210905,-0.413316
...,...,...,...,...,...,...
586,1.036538,17.378448,-30.725098,0.057484,0.359940,0.028049
587,0.490712,17.385994,-29.716721,0.066582,0.203006,-3.622377
588,0.013473,20.706028,4.450165,0.041978,0.222307,3.928732
589,0.019920,20.732220,3.630352,0.034516,0.095130,-13.776512


In [368]:
for index, spot in spot_df.iterrows():
    print(spot['fit_tth'])
    break

17.398186127287


In [362]:
test.phases['LiNbO3 161'].reflections['tth']

array([12.7748426 , 17.68332873, 18.51577704, 21.17046292, 21.35159026,
       22.55932754, 25.71200336, 28.26747365, 28.67648414, 29.34097955,
       30.37171522, 31.87280658, 32.35925707, 35.80628774, 37.21382834,
       37.31890625, 37.53841721, 38.99336444, 39.09762864, 39.80823438,
       40.60312027, 41.78118066, 41.9687455 ])

In [369]:
test.phases['Calcite'].reflections['tth']

array([12.31247529, 15.65239297, 16.71419617, 19.07696091, 20.84954958,
       22.76797891, 24.77004242, 24.96244369, 25.4671882 , 29.46215981,
       29.86972909, 31.45205614, 31.60676905, 31.78607241, 32.59335273,
       33.35903409, 33.79787873, 35.47694933, 35.96105953, 37.1795713 ,
       39.09645041, 41.01858787, 41.99202911])

In [377]:
def vprint(message):
    global _verbose
    if _verbose:
        print(message)

def some_func(verbose=False):
    global _verbose
    _verbose = verbose

    # Do something

    vprint('Did something!')


In [379]:
some_func()

In [161]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(blob_int, '.-', label='data')
ax.plot(PeakModel.multi_2d([blob_x, blob_y], *p0), '--', label='guess')
#ax.plot(PeakModel.multi_2d([blob_x, blob_y], *popt), '-', label='fit')
ax.legend()
plt.show()

In [121]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(blob_int, '.-', label='data')
ax.plot(PeakModel.multi_2d([blob_x, blob_y], *p0), '--', label='guess')
ax.plot(PeakModel.multi_2d([blob_x, blob_y], *popt), '-', label='fit')
ax.legend()
plt.show()

In [993]:
'''PeakModel = GaussianFunctions()

p0 = [np.median(image)]
for i, spot in enumerate(spots):
    p0.append(image[*spot]) # amp
    p0.append(x_coords[*spot]) # x0
    p0.append(y_coords[*spot]) # y0
    p0.append(0.05) # sigma_x
    p0.append(0.05) # sigma_y
    p0.append(0) # theta

bounds = generate_bounds(p0[1:], PeakModel.func_2d, tth_step=None, chi_step=None)
bounds[0].insert(0, -np.min(image))
bounds[1].insert(0, np.max(image))

popt, pcov = curve_fit(PeakModel.multi_2d, [fit_x, fit_y], fit_int, p0=p0, bounds=bounds)
r_squared = compute_r_squared(fit_int, PeakModel.multi_2d([fit_x, fit_y], *popt))
print(r_squared)''';

0.8956584164583816


In [978]:
full_popt.reshape(len(full_popt) // 6, 6)[:, 0]

array([0.00158355, 0.03818797, 0.00267062, 0.00816959, 0.00102969,
       0.00037197, 0.00001403, 0.01724101, 0.00600636, 0.009431  ,
       0.00014443, 0.0087252 , 0.01483163, 0.08065351, 0.09964789,
       0.00597524, 0.29795512, 0.00000342, 0.0406762 , 0.00065487])

In [910]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(fit_int, '.-', label='data')
ax.plot(PeakModel.multi_2d([fit_x, fit_y], *p0), '--', label='guess')
ax.plot(PeakModel.multi_2d([fit_x, fit_y], *popt), '-', label='fit')
ax.legend()
plt.show()

In [924]:
fit_img = PeakModel.multi_2d([x_coords.ravel(), y_coords.ravel()], *popt)
test.map.plot_image(fit_img.reshape(test.calibrated_shape[::-1]))
test.map.plot_image(image)

In [918]:
new_popt = popt[1:].reshape(len(popt[1:]) // 6, 6)
trimmed_popt = new_popt[new_popt[:, 0] > np.median(image[image != 0]) + 3 * bkg_noise]

In [925]:
fit_img = PeakModel.multi_2d([x_coords.ravel(), y_coords.ravel()], popt[0], *trimmed_popt.ravel())
test.map.plot_image(fit_img.reshape(test.calibrated_shape[::-1]))

In [979]:
new_popt[:, 0]

array([0.09287075, 0.09678619, 0.03696277, 0.03050774, 0.01765627,
       0.00980462, 0.00583943, 0.00717602, 0.0091646 , 0.00820964,
       0.00620072, 0.00364772, 0.00703682, 0.0036708 , 0.00389725,
       0.01746228, 0.00269509, 0.00600671, 0.00694821, 0.00357511])

In [931]:
new_p0 = np.asarray(p0[1:]).reshape(len(p0[1:]) // 6, 6)
trimmed_p0 = new_p0[new_p0[:, 0] > np.median(image[image != 0]) + 3 * bkg_noise]

In [933]:
trimmed_p0.shape

(19, 6)

In [854]:
img_model = None

peak_func = GaussianFunctions().func_2d
#peak_func = LorentzianFunctions().func_2d

def const_bkg_2d(xy, c=0):
    return c

img_model = Model(const_bkg_2d, prefix='bkg_')
img_params = {'bkg_c' : np.median(image)}

for index, spot in enumerate(spots):
    img_model += Model(peak_func, prefix=f'spot{index}_')

    x0_guess = x_coords[*spot]
    x0_min = x0_guess - 20 * test.tth_resolution
    x0_max = x0_guess + 20 * test.tth_resolution
    y0_guess = y_coords[*spot]
    y0_min = y0_guess - 20 * test.chi_resolution
    y0_max = y0_guess + 20 * test.chi_resolution
    
    guess_params = {f'spot{index}_amp' : dict(value=image[*spot], min=0, max=100 * np.max(image)),
                    f'spot{index}_x0' : dict(value=x0_guess, min=x0_min, max=x0_max),
                    f'spot{index}_y0' : dict(value=y0_guess, min=y0_min, max=y0_max),
                    f'spot{index}_sigma_x' : dict(value=0.1, min=0, max=5),
                    f'spot{index}_sigma_y' : dict(value=0.1, min=0, max=5),
                    f'spot{index}_theta' : dict(value=0, min=-np.inf, max=np.inf)
                    }
    
    '''guess_params = {f'spot{index}_amp' : image[*spot],
                    f'spot{index}_x0' : x_coords[*spot],
                    f'spot{index}_y0' : y_coords[*spot],
                    f'spot{index}_sigma_x' : 0.1,
                    f'spot{index}_sigma_y' : 0.1,
                    f'spot{index}_theta' : 0,
                    }'''
    
    
    '''guess_params = {f'spot{index}_amp' : image[*spot],
                    f'spot{index}_x0' : x_coords[*spot],
                    f'spot{index}_y0' : y_coords[*spot],
                    f'spot{index}_gamma_x' : 0.1,
                    f'spot{index}_gamma_y' : 0.1,
                    f'spot{index}_theta' : 0,
                    }'''
    
    img_params.update(guess_params)
params = img_model.make_params(**img_params)

In [855]:
guess = img_model.eval(params, xy=[fit_x, fit_y])
result = img_model.fit(fit_int, params, xy=[fit_x, fit_y])
print(result.rsquared)
print(np.sum(result.residual))

0.9149137841535736
-5.037434774868288e-08


In [856]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(fit_int, '.-', label='data')
ax.plot(guess, '--', label='guess')
ax.plot(result.best_fit, '-', label='fit')
ax.legend()
plt.show()

In [857]:
fit_img = img_model.eval(result.params, xy=[x_coords, y_coords])
test.map.plot_image(fit_img)

In [859]:
test.map.plot_image(image)

In [54]:
plot_value = []

for blob_df in blob_df_lst:
    if False:
        val = np.mean(blob_df['significance'])
        if type(val) != float:
            val = val[0]
    if True:
        val = np.sum(blob_df['intensity'])

    if False:
        if len(blob_df) > 0:
            #val = np.median(blob_df['max_coords'][0][0])
            val = np.median(blob_df['center_of_mass'][0][0])
        else:
            val = 0
    plot_value.append(val)

plot_value = np.asarray(plot_value).reshape(*test.map.map_shape)

In [55]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(plot_value)
fig.colorbar(im, ax=ax)
plt.show()

In [336]:
blob_df_lst[2781]

,intensity,area,significance,value,mask,max_coords,center_of_mass
0,1.985716,[0.16541614385817305],[12.00436703572288],"[0.009685349, 0.013553033, 0.0125740105, 0.007...","[[921, 274], [921, 275], [921, 276], [921, 277...","[17.710014004511, -10.468320132226474]","[17.659259707747665, -10.606500874298334]"
1,3.648422,[0.30530510502218355],[11.950085934350302],"[0.0063161175, 0.005329996, 0.011383479, 0.011...","[[896, 272], [896, 273], [896, 274], [896, 275...","[17.629212568346226, -9.458738531036722]","[17.60407915858766, -9.536232307632876]"
2,2.500236,[0.21034398539989904],[11.886416757096823],"[0.012599019, 0.013407176, 0.012197853, 0.0115...","[[632, 272], [632, 273], [633, 269], [633, 270...","[17.58881185026384, 3.5648641243110646]","[17.57938302585813, 3.8968737640178333]"
3,2.595649,[0.21851268386203104],[11.878709829049823],"[0.012455584, 0.013387301, 0.011531679, 0.0179...","[[1118, 271], [1118, 272], [1118, 273], [1119,...","[17.508010414099072, -20.26126166376705]","[17.586141508168705, -20.638710265564256]"
4,5.208407,[0.4421308042628946],[11.780239863369607],"[0.013050501, 0.009287628, 0.011285164, 0.0147...","[[384, 275], [385, 272], [385, 273], [385, 274...","[17.64941292738742, 15.5788851784691]","[17.61589939419721, 15.974939777517726]"


In [405]:
elements_folder = f'output_tiff_scan2D_{scanid}_xs_sum8ch\\'

element_maps = {}
for element in os.listdir(f'{filedir}{elements_folder}'):
    el_im = io.imread(f'{filedir}{elements_folder}{element}')
    element_maps[str(element[:-5])] = el_im

In [406]:
key = 'detsum_Au_L'
#interactive_dynamic_2d_plot(proc_map, range(121), range(121), img_vmin=10, img_vmax=100, display_map=element_maps['detsum_Pt_L'], map_vmin=1e4, map_vmax=1.1e4)
interactive_dynamic_2d_plot(test.map.images, range(test.map.images.shape[0]), range(test.map.images.shape[1]), display_map=element_maps[key], display_title=key)

KeyError: 'detsum_Au_L'